In [ ]:
import cv2
import numpy as np
from PIL import Image
from adafruit_servokit import ServoKit
import threading
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize ServoKit for controlling servos on 16 channels
kit = ServoKit(channels=16)
# Set initial pan and tilt angles
pan = 20
tilt = 20

# Set initial position for servo motors
kit.servo[0].angle = pan
kit.servo[1].angle = tilt


# Function to convert BGR image to JPEG format
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])


# Display widgets
FGmaskComp_img = widgets.Image(format='jpeg', width=320, height=240)
frame_img = widgets.Image(format='jpeg', width=320, height=240)
display_img = widgets.HBox([FGmaskComp_img, frame_img])
output = widgets.Output()

display(display_img, output)

# Camera Initialization
from picamera2 import Picamera2
import libcamera

picamera = Picamera2()
config = picamera.create_preview_configuration(main={"format": 'RGB888', "size": (320, 240)},
                                               raw={"format": "SRGGB12", "size": (1920, 1080)})
config["transform"] = libcamera.Transform(hflip=False, vflip=True)
picamera.configure(config)
picamera.start()


# Function for video display
def video_display(stop_event):
    global pan
    global tilt

    width, height = 320, 240  # Example resolution

    try:
        while not stop_event.is_set():
            frame = picamera.capture_array()

            # Display frame
            frame_img.value = bgr8_to_jpeg(frame)
            with output:
                clear_output(wait=True)
                display(frame_img)

            time.sleep(0.03)
    except Exception as e:
        print(f"ビデオ表示中のエラー: {e}")


# Function to manually stop the video display
def stop_video_display():
    stop_event.set()
    video_thread.join()


# Callback function for pan slider
def on_pan_change(change):
    global pan
    pan = change['new']
    kit.servo[0].angle = 180 - pan


# Callback function for tilt slider
def on_tilt_change(change):
    global tilt
    tilt = change['new']
    kit.servo[1].angle = 180 - tilt


# Create sliders for pan and tilt
pan_slider = widgets.IntSlider(value=20, min=0, max=180, step=1, description='Pan')
tilt_slider = widgets.IntSlider(value=20, min=0, max=180, step=1, description='Tilt')

# Link sliders to callback functions
pan_slider.observe(on_pan_change, names='value')
tilt_slider.observe(on_tilt_change, names='value')

# Display sliders
display(pan_slider, tilt_slider)

# Start the video display in a separate thread
stop_event = threading.Event()
video_thread = threading.Thread(target=video_display, args=(stop_event,))
video_thread.start()

# Button to stop video display
stop_button = widgets.Button(description="Stop Video Display")
stop_button.on_click(lambda x: stop_video_display())
display(stop_button)
